In [16]:
import cv2
import numpy as np
import math
from collections import defaultdict

In [17]:

class BubbleTracker:
    def __init__(self, max_distance=50):
        self.next_id = 0
        self.bubbles = {}  # id -> (cx, cy)
        self.history = defaultdict(list)  # id -> list of (frame, cx, cy, area)
        self.max_distance = max_distance

    def update(self, detections, frame_num):
        updated_ids = set()
        new_bubbles = {}

        for (cx, cy, area) in detections:
            matched_id = None
            min_dist = self.max_distance

            for bubble_id, (prev_cx, prev_cy) in self.bubbles.items():
                dist = math.hypot(cx - prev_cx, cy - prev_cy)
                if dist < min_dist:
                    matched_id = bubble_id
                    min_dist = dist

            if matched_id is not None:
                # Existing bubble
                new_bubbles[matched_id] = (cx, cy)
                self.history[matched_id].append((frame_num, cx, cy, area))
                updated_ids.add(matched_id)
            else:
                # New bubble
                bubble_id = self.next_id
                self.next_id += 1
                new_bubbles[bubble_id] = (cx, cy)
                self.history[bubble_id].append((frame_num, cx, cy, area))
                updated_ids.add(bubble_id)

        self.bubbles = {k: v for k, v in new_bubbles.items() if k in updated_ids}

    def get_tracks(self):
        return self.history


In [20]:


video_path = "input_files/1mm - Trim.mp4"
output_path = "video_out/output.mp4"
cap = cv2.VideoCapture(video_path)

crop_x = (300,720)
crop_y = (300, 1100)
width = crop_x[1] - crop_x[0]
height = crop_y[1] - crop_y[0]

fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

if not cap.isOpened():
    print("Error opening video file")
    exit()


frame_num = 0
tracker = BubbleTracker(max_distance=100)


while True:
    frame_num += 1

    ret, frame = cap.read()
    if not ret:
        break  # End of video

    # Crop the frame
    cropped_frame = frame[crop_y[0]:crop_y[1], crop_x[0]:crop_x[1]]

    # grayscale
    gray_frame = cv2.cvtColor(cropped_frame, cv2.COLOR_BGR2GRAY)

    blurred_frame = cv2.GaussianBlur(gray_frame, (11,11), 0)

    # Apply Canny edge detection
    edges = cv2.Canny(blurred_frame, 10, 15)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    edges = cv2.dilate(edges, kernel, iterations=1)

    # cv2.imwrite(f"edges//edge{frame_num}.jpg", edges)


    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    detected_bubbles = []
    
    # Initialize the bubble tracker
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < 1500:
          continue

        #approx polygon
        epsilon = 0.005 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)

        #draw the polygon
        cv2.polylines(cropped_frame, [approx], isClosed=True, color=(0, 255, 0), thickness=2)

        # Compute center of the polygon to place text
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
        else:
            cx, cy = approx[0][0]  # fallback to first vertex

        # Collect detection
        detected_bubbles.append((cx, cy, area))
        
        # Draw area text
        cv2.putText(cropped_frame, f"{int(area)}", (cx, cy),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        
    
    tracker.update(detected_bubbles, frame_num)
    for bubble_id, history in tracker.get_tracks().items():
        if history and history[-1][0] == frame_num:
            _, cx, cy, area = history[-1]
            cv2.putText(cropped_frame, f"ID:{bubble_id}", (cx + 25, cy + 25), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 1)
        
    out.write(cropped_frame)

    # Exit on ESC key
    if cv2.waitKey(25) & 0xFF == 27:
        break
    
    

cap.release()
out.release()
cv2.destroyAllWindows()
